In [2]:
import pandas as pd
import numpy as np

# Remplacer 'chemin_du_fichier' par le chemin réel du fichier CSV
df = pd.read_csv('C:/Users/33640/OneDrive/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

print(df.columns.tolist())

['ticker', 'open', 'high', 'low', 'close', 'volume', 'OPCL', 'pvCLCL', 'prevAdjClose', 'SPpvCLCL', 'sharesOut', 'PERMNO', 'SICCD', 'PERMCO', 'prevRawOpen', 'prevRawClose', 'prevAdjOpen']


In [3]:
import pandas as pd
import ast

# Fonction pour convertir en toute sécurité une chaîne de caractères en liste
def safe_literal_eval(s):
    try:
        # Tente de convertir la chaîne en liste
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # Si une erreur se produit, retourne une valeur par défaut, par exemple une liste vide
        return []


# Appliquer la fonction de conversion aux colonnes 'open' et 'close'
df['open'] = df['open'].apply(safe_literal_eval)
df['close'] = df['close'].apply(safe_literal_eval)

# Calculer les retours pour chaque ligne
df['return'] = df.apply(lambda row: [(c - o) / o for o, c in zip(row['open'], row['close'])], axis=1)

# Afficher les premières lignes pour vérifier
print(df.head())

  ticker                                               open  \
0     AA  [82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0,...   
1    ABM  [20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25,...   
2    ABT  [35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34...   
3    ADI  [93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5,...   
4    ADM  [12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, ...   

                                                high  \
0  [83.5625, 81.8125, 86.5, 86.375, 86.875, 87.25...   
1  [20.625, 20.375, 20.25, 20.375, 20.375, 20.25,...   
2  [36.0, 34.75, 34.3125, 35.25, 36.25, 36.0625, ...   
3  [93.875, 91.5, 88.25, 87.625, 88.5, 94.75, 94....   
4  [12.0625, 12.1875, 11.875, 11.875, 12.0, 12.18...   

                                                 low  \
0  [80.375, 80.3125, 81.0, 84.8125, 84.5625, 84.3...   
1  [20.0, 20.0, 20.0, 20.0625, 20.0625, 20.0, 20....   
2  [34.75, 33.75, 33.5625, 33.8125, 34.5, 34.875,...   
3  [88.0, 85.5625, 83.1875, 83.25, 82.625, 89.25,...   
4  [

In [4]:
new_df = df[['ticker', 'return']]

new_df.shape

(664, 2)

In [5]:
# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

def check_nan_inf(df):
    # Vérification des valeurs NaN
    if df.isna().any().any():
        print("Il y a des valeurs NaN dans le DataFrame")
    else:
        print("Il n'y a pas de valeurs NaN dans le DataFrame")

def remove_rows_with_nan(df):
    return df.dropna()

df_cleaned = remove_rows_with_nan(returns_df)

check_nan_inf(df_cleaned)

df_cleaned.head()

Il n'y a pas de valeurs NaN dans le DataFrame


,ticker,return_0,return_1,return_2,return_3,return_4,return_5,return_6,return_7,return_8,...,return_5521,return_5522,return_5523,return_5524,return_5525,return_5526,return_5527,return_5528,return_5529,return_5530
0,AA,-0.012957,0.010093,0.048780,-0.011645,-0.015988,-0.031564,0.022866,-0.005236,-0.018045,...,0.028129,0.023511,0.055576,0.041806,0.006102,0.025013,-0.001518,0.008456,-0.007875,0.007099
1,ABM,-0.009146,0.012422,-0.006173,0.000000,0.003096,0.000000,-0.009259,0.003135,-0.028213,...,-0.016850,-0.021608,0.030867,0.013885,0.012927,-0.000724,-0.002186,-0.007579,-0.001969,0.006406
2,ABT,-0.007092,-0.012704,0.011173,0.033088,0.028986,-0.020833,0.010909,-0.021544,-0.009174,...,-0.003126,-0.008957,-0.005514,0.022888,-0.003509,0.011729,-0.007840,0.005126,-0.002899,-0.001844
3,ADI,-0.035428,-0.043296,0.014599,-0.027338,0.034226,0.049306,-0.031417,0.035294,-0.020235,...,-0.005902,0.008901,0.011551,0.003729,-0.000810,0.014219,-0.011563,0.005672,-0.006932,0.005837
4,ADM,0.000000,0.005291,-0.015789,0.010753,0.005263,-0.005208,-0.015707,0.021164,0.000000,...,-0.009259,-0.007572,0.004027,-0.003547,0.009877,0.008087,0.014091,-0.000597,-0.005060,0.013343


In [6]:
for i in range(20):
    start_col = i * 250 + 1  # Commence à partir de la deuxième colonne
    end_col = start_col + 250

    # Vérifie si la fin de la tranche dépasse le nombre de colonnes
    if end_col > len(df_cleaned.columns):
        end_col = len(df_cleaned.columns)  # Ajuste pour ne pas dépasser

data_period = df_cleaned.iloc[:, start_col:end_col]

data_period.head()


,return_4750,return_4751,return_4752,return_4753,return_4754,return_4755,return_4756,return_4757,return_4758,return_4759,...,return_4990,return_4991,return_4992,return_4993,return_4994,return_4995,return_4996,return_4997,return_4998,return_4999
0,0.008112,0.005247,-0.029558,0.071770,-0.073543,0.024807,-0.025836,-0.053254,-0.044113,-0.011173,...,0.006702,0.003160,-0.006717,-0.014399,-0.002850,-0.005238,0.006790,-0.021874,-0.001471,-0.013705
1,0.003652,-0.002294,0.000995,0.010880,0.021526,0.015385,-0.032439,-0.043240,-0.033585,-0.023379,...,0.010963,0.002928,0.015103,0.001318,0.009752,-0.017885,0.015071,0.006753,0.003378,-0.022182
2,0.002502,0.001164,0.014554,0.038944,0.011459,0.001623,-0.000269,-0.036907,-0.025087,0.012564,...,0.004920,0.005402,0.008128,0.003097,0.000357,0.012283,-0.016918,-0.003900,-0.005336,-0.000478
3,-0.000113,-0.002658,0.001341,0.014425,-0.005112,0.024749,0.001821,-0.028989,-0.029752,0.024548,...,0.009287,0.000000,0.002035,0.017014,-0.003342,-0.011334,-0.009378,-0.009048,-0.020036,-0.002567
4,-0.001546,0.002864,0.011638,-0.002169,0.004801,0.000870,-0.006785,-0.027298,-0.019841,0.001797,...,-0.000462,0.006009,-0.008257,-0.005343,-0.002797,-0.001398,0.000935,0.007955,-0.007009,0.005400


In [7]:
# Supprimez la colonne 'ticker' pour ne garder que les données numériques
returns_only = data_period

returns_transposed = returns_only.transpose()

# Calculer la matrice de corrélation sur les actifs transposés
correlation_matrix = returns_transposed.corr(method='pearson')


# Afficher la matrice de corrélation
print(correlation_matrix)

          0         1         2         3         4         5         6    \
0    1.000000  0.299607  0.327069  0.382164  0.333826  0.412230  0.103181   
1    0.299607  1.000000  0.291538  0.223630  0.226859  0.334912  0.157647   
2    0.327069  0.291538  1.000000  0.460261  0.337487  0.561283  0.344235   
3    0.382164  0.223630  0.460261  1.000000  0.394347  0.500258  0.076093   
4    0.333826  0.226859  0.337487  0.394347  1.000000  0.407160  0.286774   
..        ...       ...       ...       ...       ...       ...       ...   
659  0.557932  0.394316  0.591948  0.614395  0.423729  0.743285  0.188657   
660  0.403585  0.121729  0.349916  0.405667  0.406284  0.531886  0.152259   
661  0.342843  0.169579  0.295753  0.355419  0.425380  0.474388  0.221104   
662  0.088565  0.156215  0.409285  0.194597  0.153418  0.321535  0.343564   
663  0.136778  0.081944  0.171438  0.130567  0.201059  0.322670  0.199445   

          7         8         9    ...       654       655       656  \
0  

In [8]:
def signed_adjency(mat):
    '''
    L'idée est ici, à partir d'une matrice de corrélation mat, de renvoyer deux matrices 
    A_positive et A_negative qui correspondraient aux matrices des corrélations positives et négatives 
    associées  
    '''

    A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
    A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
    
    return A_pos, A_neg

signed_adjency(correlation_matrix)

(          0         1         2         3         4         5         6    \
 0    1.000000  0.299607  0.327069  0.382164  0.333826  0.412230  0.103181   
 1    0.299607  1.000000  0.291538  0.223630  0.226859  0.334912  0.157647   
 2    0.327069  0.291538  1.000000  0.460261  0.337487  0.561283  0.344235   
 3    0.382164  0.223630  0.460261  1.000000  0.394347  0.500258  0.076093   
 4    0.333826  0.226859  0.337487  0.394347  1.000000  0.407160  0.286774   
 ..        ...       ...       ...       ...       ...       ...       ...   
 659  0.557932  0.394316  0.591948  0.614395  0.423729  0.743285  0.188657   
 660  0.403585  0.121729  0.349916  0.405667  0.406284  0.531886  0.152259   
 661  0.342843  0.169579  0.295753  0.355419  0.425380  0.474388  0.221104   
 662  0.088565  0.156215  0.409285  0.194597  0.153418  0.321535  0.343564   
 663  0.136778  0.081944  0.171438  0.130567  0.201059  0.322670  0.199445   
 
           7         8         9    ...       654       655   

In [9]:
import sys

sys.path.append('C:/Users/33640/OneDrive/Documents/GitHub/Portfolio_clustering_project')  # Ajoute le chemin parent

from signet.cluster import Cluster 

In [10]:
from scipy import sparse

def apply_SPONGE(correlation_matrix, k): 

    '''
    IDÉE : étant donné une matrice de correlation obtenue à partir d'une base de donnée et de la similarité de pearson, renvoyer un vecteur associant 
           à chaque actif le numéro du cluster auquel il appartient une fois qu'on lui a appliqué SPONGE (à partir du package signet)

    PARAMS : 

    - correlation_matrix : a square dataframe of size (number_of_stocks, number_of_stocks)
    - k : the number of clusters to identify. If a list is given, the output is a corresponding list

    RETURNS : array of int, or list of array of int: Output assignment to clusters.

    '''
    
    ## On respecte le format imposé par signet. Pour cela il faut changer le type des matrices A_pos et A_neg, qui ne peuvent pas rester des dataframes 

    A_pos, A_neg = signed_adjency(correlation_matrix)

    A_pos_sparse = sparse.csc_matrix(A_pos.values)
    A_neg_sparse = sparse.csc_matrix(A_neg.values)

    data = (A_pos_sparse, A_neg_sparse)

    cluster = Cluster(data)

    ## On applique la méthode SPONGE : clusters the graph using the Signed Positive Over Negative Generalised Eigenproblem (SPONGE) clustering.

    return cluster.SPONGE(k )

In [14]:
result = apply_SPONGE(correlation_matrix, k=30)

print(result)


[ 8  8 15  8  8 15  4  8 13  4 15 15 15 15  8  8 28 15  8  8 15  8 15  8
  8 28  8  8  8  2 15  8 28  8 13 15  8  8  4 15 13  4 28  8  8  0  4  8
 15 15 15  8 15  8 15  8  8  8 28 15  8 15 28  8 15  8  8  4 10  7  8 15
  8 15  8  2  8 15 15 15 15  8  8 28  8 15  8 15  8 15 26  8 28 15 15 15
  8  8 15  0 15  2  8  8 15  8  8  4 17 15  8  2  8 15 15 15  2  4 28  8
  6  2  8 15  8  8  8 28 15  8  4  8  3  4  8  8 15  8  8  0  6 15  0 15
  8 15  8 15 28 15  2  3  0  4  4  2  8  8  8 15  4 15 28  4 15  8 15  8
  8 15  2  2 28 28  2  8 28  8  4  0 15 15 15 15 15  8 15  8  8  8 15  8
  2 15  8  4  8  8  8 15 15  8 28 28  8  0  6 15 15  8  0  8  8  8  8 15
  8  8 15 15  8  6 15  8  8  8  8  8  8 28  8  8 15  2  8 28 15  4 15 15
  0 28  0 26  8  8  8 15  2 15 15 28  8 15  8  4 15  8 15  0  8  4  8 15
  8 15 15  8  8  2  8  8  3 28 15  8  2  8  8 15  8 15  8  8  8 15  6 15
  8  8 28  4  8 15 15 15  8 28 16  8  3  2  8  6  1  8 15 15 15 15  8  8
  4  8  8 28  5  8 28  8 28 28  8 15  8  8  8 14 15

In [15]:
l=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in result:
    for j in range(20):
        if i==j:
            l[i]+=1

print(l)

[22, 4, 41, 9, 37, 1, 13, 1, 263, 1, 4, 1, 1, 4, 4, 160, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
data_period=[]
result=[]
for i in range(28):
    start_col = i * 200 + 1  # Commence à partir de la deuxième colonne
    end_col = start_col + 200

    # Vérifie si la fin de la tranche dépasse le nombre de colonnes
    if end_col > len(df_cleaned.columns):
        end_col = len(df_cleaned.columns)  # Ajuste pour ne pas dépasser
    
    data_period.append(df_cleaned.iloc[:, start_col:end_col]) 

    returns_transposed = data_period[i].transpose()

    # Calculer la matrice de corrélation sur les actifs transposés
    correlation_matrix = returns_transposed.corr(method='pearson')
    result.append(apply_SPONGE(correlation_matrix, k=10))



In [98]:
l=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in resultat:
    for j in range(10):
        if i==j:
            l[i]+=1

print(l)

[508, 5, 15, 14, 18, 31, 14, 19, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# Supposons que vous avez une liste de listes, où chaque sous-liste est un des 28 clusterings
# Chaque sous-liste contient 632 éléments avec des valeurs de 0 à 9
clusterings = result  # Remplacez ceci par vos données de clustering

# Convertir les résultats de clustering en une matrice de co-occurrence
n_data = 632
co_occurrence_matrix = np.zeros((n_data, n_data))

for clustering in clusterings:
    for i in range(n_data):
        for j in range(n_data):
            if clustering[i] == clustering[j]:
                co_occurrence_matrix[i, j] += 1

# Convertir en matrice de similarité
similarity_matrix = co_occurrence_matrix / len(clusterings)

# Convertir la matrice de similarité en matrice de distance
distance_matrix = 1 - similarity_matrix

# Convertir la matrice de distance en format condensé pour la fonction linkage
# car linkage attend une matrice de distance condensée pour les données non euclidiennes
tri_upper_idx = np.triu_indices(n_data, k=1)
distance_condensed = distance_matrix[tri_upper_idx]

# Effectuer le clustering hiérarchique
Z = linkage(squareform(distance_condensed), method='average')

# Former des clusters
# Vous pouvez ajuster le 't' pour obtenir le nombre de clusters souhaité
clusters = fcluster(Z, t=10, criterion='maxclust')

# Afficher les clusters
print(clusters)

[ 1  1  7  2  2  2  8  2  5  8  2  2  2  2  1  1  9  2  2  2  2  2  2  2
  2  2  2  1  2  6  2  2  9  2  5  2  1  2  8  2  5  8  9  2  2  4 10  1
  2  2  2  1  2  2  7  2  2  2  9  7  2  7  9  1  1  2  1 10  3  3  2  2
  2  7  1  5  1  2  2  2  2  2  2  9  2  7  2  2  2  2  5  1  7  2  2  2
  1  2  4  4  7  6  1  2  7  2  1  8  3  2  2  6  2  2  2  7  6 10  9  2
  1  6  1  1  2  2  1  9  2  2 10  2  3  8  2  2  6  1  2  4  2  2  4  2
  1  4  2  2  9  2  6  3  4  8  8  6  1  1  1  2  8  2  9  8  2  1  2  2
  2  1  6  6  9  9  6  1  9  2  8  4  2  2  2  2  2  2  2  1 10  1  1  1
  1  2  2  8  1  2  2  2  2  2  9  9  1  4  2  2  2  1  1  1  2  2  1  2
  1  2  2  2  2  4  7  1  2  2  1  2  2  5  1  2  1  6  2  9  2  8  1  2
  4  9  4  5  2  1  1  2  6  2  2  9  2  7  1  7  7  1  2  4  2  8  2  2
  1  2  2  2  1  6  2  2  3  2  2  2  1  2  2  7  1  2  2  1  2  7  2  2
  1  1  9  7  2  2  7  2  7  9  3  2  3 10  2  2  3  1  7  2  7  2  2  2
  8  2  2  9 10  2  9  1  9  9  2  2  2  2  2  3  2

C:\Users\33640\AppData\Local\Temp/ipykernel_3188/3948724184.py:31: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(squareform(distance_condensed), method='average')


In [22]:
l=[0,0,0,0,0,0,0,0,0,0,0]
for i in clusters:
    for j in range(1,11):
        if i==j:
            l[i]+=1

print(l)

[0, 113, 292, 42, 25, 12, 38, 30, 25, 43, 12]
